In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
dataset = pd.read_csv('census.csv')

## part1\. AM simples

A amostragem aleatória simples é o processo mais elementar e frequentemente utilizado. Corresponde a uma amostra de elementos retirados ao acaso da população, isto é, cada indivíduo é escolhido completamente ao acaso e cada membro da população tem a mesma probabilidade se ser incluído na amostra

In [3]:
def amostra_aleatoria_simples(dataset, amostras):
  return dataset.sample(n = amostras)


In [4]:
df_amostra_aleatoria_simples = amostra_aleatoria_simples(dataset, 100)

## part2\. AM sistematica.

Na amostra sistematica, é feita uma seleção aleatória do primeiro elemento para a amostra e logo se selecionam os itens subsequentes utilizando intervalos fixos ou sistemáticos até chegar ao tamanho da amostra desejada. Por exemplo, em uma linha de produção, podemos, **a cada dez itens produzidos**, retirar um para pertencer a uma amostra da produção diária.

In [5]:
def amostra_sistematica(dataset, amostras):
  intervalo = len(dataset) // amostras
  random.seed(1)
  inicio = random.randint(0, intervalo)
  indices = np.arange(inicio, len(dataset), step= intervalo)
  amostra_sistematica = dataset.iloc[indices]
  return amostra_sistematica

In [6]:
df_amostra_sistematica = amostra_sistematica(dataset, 100)

## part3\. AM de Agrupamento

A amostragem de agrupamento é um tipo de método de amostragem em que a população respondente é dividida em agrupamentos iguais. Os clusters são identificados e incluídos em uma amostra com base na definição de parâmetros demográficos, como idade, local, sexo, etc; o que torna extremamente fácil para um criador de pesquisa derivar inferência efetiva do feedback. Por exemplo, se o FDA quiser coletar dados sobre efeitos colaterais adversos dos medicamentos; eles podem dividir os EUA do continente em grupos distintos, como estados. Os estudos de pesquisa são administrados aos entrevistados nesses clusters. Esse tipo de geração de uma amostra facilita a coleta de dados e fornece informações de fácil consumo e ação.

In [14]:
def amostra_agrupamento(dataset, numero_grupos):
  intervalo = len(dataset) / numero_grupos

  grupos = []
  id_grupos = 0
  contagem = 0 

  for _ in dataset.iterrows():
    grupos.append(id_grupos)
    contagem += 1
    if contagem > intervalo:
      contagem = 0
      id_grupos += 1

  dataset['grupo'] = grupos
  grupo_selecionado = random.randint(0, numero_grupos) 
  return dataset[dataset['grupo'] == grupo_selecionado]

In [15]:
df_amostra_agrupamento = amostra_agrupamento(dataset, 100)

## part4\. AM estratificada

Permite a obtenção de amostras de menor dimensão quando na presença de amostras com grande dispersão (desvio padrão elevado). A mmostra estratificada encontra a quantidade desejada na mesma distribuição original. 

In [37]:
from sklearn.model_selection import StratifiedShuffleSplit

* Representatividade (distribuição original)

In [38]:
dataset['income'].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

In [39]:
7841 / len(dataset), 24720 / len(dataset)

(0.2408095574460244, 0.7591904425539756)



*   Seleção quantidade



In [40]:
100 / len(dataset) # 100 pessoas 

0.0030711587481956942

* StratifiedShuffleSplit

In [41]:
split = StratifiedShuffleSplit(test_size=0.1) # Porcentagem de amostra
for x,y in split.split(dataset, dataset['income']):
  df_x = dataset.iloc[x]
  df_y = dataset.iloc[y]

* Amostra original

In [42]:
dataset['income'].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

* Amostra estratificada da quantidade desejada na mesma distribuição original. 

In [43]:
df_y['income'].value_counts()

 <=50K    2473
 >50K      784
Name: income, dtype: int64

In [44]:
def amostra_estratificada(dataset, percentual):
  split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
  
  for _, y in split.split(dataset, dataset['income']):
    df_y = dataset.iloc[y]

  return df_y

In [45]:
df_amostra_estratificada = amostra_estratificada(dataset, 0.0030711587481956942)

## part5\. AM de reservatorio 

A amostragem de reservatório é uma família de algoritmos randomizados para escolher aleatoriamente k amostras de uma lista de **N itens**, onde **N** é um número muito grande ou desconhecido. Normalmente n é grande o suficiente para que a lista não caiba na memória principal.(Por exemplo) na fabrica de refrigerante todas as garrafinhas **(N itens)** passando na esteira deve ter a mesma chance de ser escolhida durante o processo de amostra.

In [28]:
def amostra_reservatorio(dataset, amostras):
  stream = []
  for i in range(len(dataset)):
    stream.append(i)

  i = 0
  tamanho = len(dataset)

  reservatorio = [0] * amostras
  for i in range(amostras):
    reservatorio[i] = stream[i]

  while i < tamanho:
    j = random.randrange(i + 1)
    if j < amostras:
      reservatorio[j] = stream[i]
    i += 1

  return dataset.iloc[reservatorio]        

In [36]:
df_amostra_reservatorio = amostra_reservatorio(dataset, 100)

## final\. Técnicas de amostragem

O ideal é usar a amostra que mais se aproximar da media total. para garantir a representatividade de todos.

* media total

In [30]:
dataset['age'].mean()

38.58164675532078

* media amostra simples

In [31]:
df_amostra_aleatoria_simples['age'].mean()

37.55

* media amostra sistematica

In [32]:
df_amostra_sistematica['age'].mean()

37.57

media amostra agrupada

In [33]:
df_amostra_agrupamento['age'].mean()

39.47546012269939

media amostra estratificada

In [34]:
df_amostra_estratificada['age'].mean()

38.58213079521032

media amostra reservatorio

In [46]:
df_amostra_reservatorio['age'].mean()

38.32